In [1]:
import numpy as np
import numpy.linalg as la
# import scipy.sparse as sparse

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import timeit
import numba
from numba import jit, njit

from iteration_methods import*
from basic import*

In [2]:
#output settings
np.set_printoptions(precision=2)
%matplotlib qt 

In [3]:
@njit
def source(r, th, u):

    f = 1/np.sin(th)

    return f

@njit
def source_1(r, th, u, params = (1, 1, 0)):

    # f1 = r**2/u

    sigma, s, uc = params

    if (u-uc) > 0.:
        
        f1 = sigma*s**2*(u-uc)**(2*sigma-1)

    else:

        f1=f1*0.
    
    return f1

@njit
def u_analytical(r, th):

    uan = r**2*np.sin(th) 

    return uan

In [4]:
def grad_shaf_solver(matrix, source_term, grid, init_guess=None, 
                    boundary=((0, 0), (0,0)), tolerance=1.e-8, 
                    itermax=1000, omega=1.5, params=(1, 1, 0)):

    #read the input
    M, u0, B, tol, kmax = matrix, init_guess, boundary, tolerance, itermax

    #assign the grid
    r, th = grid

    R, TH = r[0], th[:,0]

    Nr, Nth = r.shape[1]-1, th.shape[0]-1

    dr, dth = (r[0, -1] - r[0, 0])/Nr, (th[-1, 0] - th[0, 0])/Nth

    
    #assign the initial guess
    if init_guess is None:

        u0 = np.ones_like(r)

    #assign dirichlet boundary conditions
    u0[:, 0] = B[0][0]                               
    u0[:, -1] = B[0][1]
    u0[0, :] = B[1][0]
    u0[-1, :] = B[1][1]
    
    #assign the source term
    f = source_term(r, th, u0)
    
    #initial values before the iteration loop starts
    u = u0.copy()
    k = 0
    rel_diff = tol + 1
    conv_hist = []

    

    #iteration loop 
    while  k < kmax and rel_diff > tol:    

        u_next = u.copy()

        #Update the source term if it is a function of the solution
        f = source_term(r, th, u)

        #calculate the solution in the kth step
        for j in range(1, Nth):
            for i in range(1, Nr):

                u_next[j,i] = (1-omega)*u[j,i] + omega/(2*(R[i]**2*dth**2 + dr**2))*(R[i]**2*dth**2*(u[j,i+1] + u_next[j, i-1]) + 
                                                                                      dr**2*(u[j+1,i]*(1+dth/(2*np.tan(TH[j]))) + 
                                                                                     u_next[j-1,i]*(1 - dth/(2*np.tan(TH[j])))) 
                                                                                     -f[j,i]*dr**2*dth**2*R[i]**2)
                
        #calculate the L2 norm of the relative difference between the two last iterations   
        rel_diff = la.norm(u_next-u)/la.norm(u)
        
        #Save the convergence history
        conv_hist.append(rel_diff)

        #update solution for next iteration
        u = u_next

        k += 1

    return u, k, rel_diff, conv_hist

In [5]:
#Setup the parameters for the solver. Function grid() is in the basic.py module

#The grid of the problem
R, TH, r, th, dr, dth = polar_grid(th_I=np.pi/6, th_F=np.pi/3, r_I=1, r_F=2, Nr=40, Nth=40)            

#initial guess
u_guess = np.ones(r.shape)       

#extra parameters that may be needed
sigma = 1.
s = 1.6
uc = 0.5
params = (sigma , s, uc)

#source term
f = source_1(r, th, u_guess, params)      

#analytical solution for comparison
u_an = u_analytical(r, th)

#boundary conditions
boundary = ((np.sin(TH), 4*np.sin(TH)), (R**2/2., R**2*np.sqrt(3)/2))                                   

#max number of iteration
iterations = 2000                              

#desired tolerance
tolerance = 1.e-10                               

#relaxation parameter for SOR method
omega_opt = 2/(1+np.sin(np.pi*max(dr, dth)))     

TypingError: Failed in nopython mode pipeline (step: nopython mode backend)
[1m[1mInvalid use of Function(<class 'bool'>) with argument(s) of type(s): (array(bool, 2d, C))
 * parameterized
[1mIn definition 0:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 1:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 2:[0m
[1m    TypingError: [1mInvalid use of Function(<built-in function truth>) with argument(s) of type(s): (array(bool, 2d, C))
 * parameterized
[1mIn definition 0:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 1:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 2:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 3:[0m
[1m    All templates rejected without literals.[0m
[1mIn definition 4:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 5:[0m
[1m    All templates rejected without literals.[0m
[1mThis error is usually cau

In [6]:
#jit the solver function
grad_shaf_solver = jit(nopython=True)(grad_shaf_solver)

In [7]:
#Calculate, time and qualify the solution

start = timeit.default_timer()

u, k, rel_diff, conv_hist = grad_shaf_solver(np.eye(99), source_1, (r, th), init_guess=u_guess, boundary=boundary,                                                                         tolerance=tolerance, itermax=iterations, omega=omega_opt, params=params)

elapsed = timeit.default_timer() - start

#Relative error to analytical
error_to_an = la.norm(u - u_an, 2)/la.norm(u_an, 2)

#print solution
print('SOR solver \nNumber of iterations: {:} \nLast Relative difference: {:1.3E}' 
      '\nError to analytical: {:1.3E}s \nElapsed time: {:1.2E}s '.format(k, rel_diff, error_to_an, elapsed))
print('-'*50)

NameError: name 'boundary' is not defined

In [8]:
#Plot results. Function plot() is in basic.py module.
plot(r*np.cos(th), r*np.sin(th), u, u_an=u_an, conv_hist=conv_hist)

NameError: name 'u' is not defined

In [13]:
bla = np.arange(10) - 5
(np.sign(bla))

array([-1, -1, -1, -1, -1,  0,  1,  1,  1,  1])